In [1]:
!pip install TurkishStemmer
! pip install git+https://github.com/ftkurt/python-syllable.git@master
from syllable import Encoder
from google.colab import files
import pandas, re, random, numpy
from TurkishStemmer import TurkishStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ftkurt/python-syllable.git (to revision master) to /tmp/pip-req-build-av4apgk4
  Running command git clone --filter=blob:none --quiet https://github.com/ftkurt/python-syllable.git /tmp/pip-req-build-av4apgk4
  Resolved https://github.com/ftkurt/python-syllable.git to commit e0a4d99e55a2e27c87b2a5cec9532dcb14e3817d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.3/399.3 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 21.1 MB/s eta 0:00:00
  Created wheel for syllable: filename=syllable-0.0.1-py3-none-any.whl size=209758 sha256=fd69d553b992bbdcb2059c727de6b25cfb61f3bb561507adcf71ba15b59cd7b2
  Stored in direct

In [2]:
vowels = "aeıioöuü"
consonants = "bcçdfgğhjklmnprsştvyz"

In [12]:
def probabilities(stems):
  encoder = Encoder(lang="tr")
  sound_prob = {}
  in_m = {}
  for word in stems:
    i = 0 #index value
    morphemes = encoder.tokenize(word).split()
    if morphemes != 1:
      if morphemes[0] not in in_m:
        in_m[morphemes[0]] = 1/(len(stems))
      else:
        in_m[morphemes[0]] += 1/(len(stems))
      #print(in_m)
      while i < (len(morphemes) - 1): #while the index is still in the bound of the word
        b = morphemes[i]
        n = morphemes[i+1]
        if n != 0: #in the cases where we have two biagrams following each other, (the initial biagram is not the last biagram of the word)
          if b in sound_prob: #if the first biagram is in the database of biagrams
            if n in sound_prob: #if the second one is in the database as well
              bInd = list(sound_prob).index(n) #find the index of the second biagram
              sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1 #and manipulate the probability value in the row of b that has the index of the following biagram
            else:
              probs = len(sound_prob) * "0" #if the second biagram does not exist in the dataset, we need to add it to the matrix and it needs a value row full of 0's (because its frequency with other sounds is 0)
              sound_prob[n] = list(probs) 
              for x in sound_prob: sound_prob[x].append(0) #every row of the matrix should be appended with 0 because the new sound that is added is also added as a new column that has never occured with the other sounds before
              sound_prob[b][-1] = 1 #the last entry of the probability of b should be altered to 1 because the new added sound appeared after that so +1 occurence
          else:
            probs = len(sound_prob) * "0" #If b does not exist in the dataset we should add it
            sound_prob[b] = list(probs)
            for x in sound_prob: sound_prob[x].append(0)
            if n in sound_prob:
              bInd = list(sound_prob).index(n)
              sound_prob[b][bInd] = int(sound_prob[b][bInd]) + 1
            else:
              sound_prob[n] = list(probs)
              sound_prob[n].append(0)
              for x in sound_prob: sound_prob[x].append(0)
              sound_prob[b][-1] = 1
        i += 1
  df = pandas.DataFrame.from_dict(sound_prob, orient="index", columns=sound_prob.keys()) #to display the matrix
  df = df.applymap(int) #to make every entry of the matrix an integer
  df = df/len(sound_prob.keys()) #to calculate the probabilities each occurence should be divided with all occurences
  return df, sound_prob, in_m

"""  with pandas.option_context('display.max_rows', None,
                        'display.max_columns', None,
                        'display.precision', 3,
                        ): print(df)"""

"  with pandas.option_context('display.max_rows', None,\n                        'display.max_columns', None,\n                        'display.precision', 3,\n                        ): print(df)"

In [13]:
def vowel_harmony(m1, m2):
    if re.search("[aı]", m1):
      m2 = re.sub("e|i|o|u|ö|ü","a", m2)
    elif re.search("[ei]", m1):
      m2 = re.sub("a|ı|o|u|ö|ü","e", m2)
    elif re.search("[ou]", m1):
      m2 = re.sub("e|i|ı|o|ö|ü","a", m2)
    elif re.search("[öü]", m1):
      m2 = re.sub("a|i|ı|o|ö|u","e", m2)
    return m2

In [14]:
def categorize(words):
  categories = {"SUBJECT": [],
            "PREDICATE": [],
            "ATTRIBUTE":[],
            "OBJECT": [],
            "ADVERBIAL MODIFIER": []}
  for w in words:
    c = random.choice(list(categories.keys()))
    if c == "PREDICATE":
      if re.search("["+vowels+"]$", w):
        if re.search("[aıou]$", w):
          w = w + "dı"
        else:
          w = w + "di"
      else:
        if re.search("[fstkçşhp]$", w):
          if re.search("[aıou]", w):
            w = w + "tı"
          else:
            w = w + "ti"
        else:
          if re.search("[aıou]", w):
            w = w + "dı"
          else:
            w = w + "di"
    elif c == "OBJECT":
      if re.search("["+vowels+"]$", w):
        if re.search("[aıou]$", w):
          w = w + "yı"
        else:
          w = w + "yi"
      else:
        if re.search("p$", w):
          if re.search("[aıou]", w):
            w = re.sub("p$", "bı", w)
          else:
            w = re.sub("p$", "bi", w)
        elif re.search("ç$", w):
          if re.search("[aıou]", w):
            w = re.sub("ç$", "cı", w)
          else:
            w = re.sub("ç$", "ci", w)
        elif re.search("t$", w):
          if re.search("[aıou]", w):
            w = re.sub("t$",  "tı", w)
          else:
            w = re.sub("t$",  "ti", w)
        elif re.search("k$", w):
          if re.search("[aıou]", w):
            w = re.sub("k$",  "ğı", w)
          else:
            w = re.sub("k$",  "ği", w)
        else:
          if re.search("[aıou]", w):
            w = w + "ı"
          else:
            w = w + "i"
    elif c == "ADVERBIAL MODIFIER":
      if re.search("[fstkçşhp]$", w):
        w = w + "ç"
      else:
        w = w + "c"
      if re.search("[aıou]", w):
        w = w + "a"
      else:
        w = w + "e"
    categories[c].append(w)
  return categories

In [15]:
def sent_generator(dic):
  subj = random.choice(list(dic["SUBJECT"]))
  pred = random.choice(list(dic["PREDICATE"]))
  attr = random.choice(list(dic["ATTRIBUTE"]))
  obje = random.choice(list(dic["OBJECT"]))
  adve = random.choice(list(dic["ADVERBIAL MODIFIER"]))
  sent_str = random.choice(["sent_str1", "sent_str2"])
  if sent_str == "sent_str1":
    sent = subj + " " + attr + " " + obje + " " + adve + " " + pred + "."
  else:
    sent = subj + " " + adve + " " + attr + " " + obje + " " + pred + "."
  return sent

In [7]:
file = files.upload() #uploading the dataset

Saving tr_TR.csv to tr_TR.csv


In [16]:
dataset = open("tr_TR.csv", "r", encoding="utf8") #opening and reading the dataset
dataset = dataset.read()
words = dataset.split("\n")

In [17]:
stems = []
for w in words[100000:150000]: #goes through all words in the dataset and makes them lower case
  w = TurkishStemmer().stem(w) #A function to get rid of the Turkish affixes
  if w in words and len(w) != 1 and w.lower() == w: #In the dataset there are some one letter words which do not exist in Turkish (i.e., "a", it is not a word in Turkish), to get rid of these letters
    if w not in stems: #After stemming we might have the same word as in the dataset. To get rid of these instances.
      stems.append(w)

print(stems)

['edip', 'edici', 'edicilik', 'edik', 'edil', 'edildik', 'edilebil', 'edilebildik', 'edilebilecek', 'edilebilen', 'edilebilerek', 'edilebilinen', 'edilebilinil', 'edilebilinmek', 'edilebilip', 'edilebilmek', 'edilebilme', 'edilecek', 'ediledur', 'edilegel', 'edilegelen', 'edilegelmek', 'edile', 'edilemedik', 'edileme', 'edilememek', 'edilememe', 'edilememezlik', 'edilemeyebil', 'edilemeyebilmek', 'edilemeyecek', 'edilemeyen', 'edilemeyerek', 'edilemeyip', 'edilemeyiş', 'edilemez', 'edilemezlik', 'edilemiyor', 'edilen', 'edilenen', 'edileniş', 'edilerek', 'edilesi', 'edileyaz', 'edilgen', 'edilgenleşen', 'edilgenleşip', 'edilgenleş', 'edilgenleşmek', 'edilgenleştirebil', 'edilgenleştiren', 'edilgenleştirerek', 'edilgenleştirilen', 'edilgenleştiril', 'edilgenleştir', 'edilgenleştirmek', 'edilgenlik', 'edilgileştiril', 'edilginleşen', 'edilginleş', 'edilginleşmek', 'edilginleştiril', 'edilginleştirmek', 'edilginlik', 'edilik', 'edililen', 'edililiyor', 'edilinebil', 'edilinen', 'edilin', 

In [18]:
df_prob, dict_prob, in_morph = probabilities(stems)

In [19]:
p_words = []
for j in range(300):
  pword = []
  i = 0
  #plen = random.choices(list(word_len.keys()), list(word_len.values()))
  plen = 7
  while i < plen:
    if pword == []:
      sound = random.choices(list(in_morph.keys()), in_morph.values())[0]
      """while sound[0] == "ğ":
        sound = random.choice(list(dict_prob.keys()))"""
      i += len(sound)
    else:
      sound = random.choices(list(dict_prob.keys()), list(df_prob[pword[-1]]))[0]
      i += len(sound)
    pword.append(sound)
  m = 1
  while m < len(pword):
    pword[m] = vowel_harmony(pword[m-1], pword[m])
    m += 1
  print(pword)
  pword = "".join(pword)
  cri = "\w*[" + vowels + "]" + "[" + vowels + "]\w*"
  if not re.search(cri, pword) and pword not in p_words:
    p_words.append(pword)

print(p_words)


['hal', 'ba', 'ya']
['e', 'te', 'fe', 'gre']
['gü', 'şe', 're', 'te']
['hi', 'fe', 'he', 'te']
['ge', 'gel', 'hiz']
['fı', 'ha', 'fal']
['e', 're', 'te', 'ges']
['ge', 'gel', 'hiz']
['ge', 'gel', 'hiz']
['ge', 'gel', 'hiz']
['ge', 'e', 're', 'hid']
['ge', 're', 'çi', 'çek']
['haz', 'haz', 'haz']
['ha', 'gaz', 'haz']
['harp', 'haz']
['e', 're', 'lekt']
['es', 'hiz', 'hiz']
['ge', 'sir', 'e', 're']
['e', 'te', 'fe', 'gre']
['fi', 're', 'fe', 'se']
['ge', 'gel', 'hiz']
['fan', 'fan', 'fan']
['fir', 'hiz', 'hiz']
['gün', 'hez', 'hiz']
['göz', 'müş', 'gü']
['hi', 'fe', 'se', 'fel']
['gö', 'nem', 'he']
['ha', 'sal', 'ga']
['fa', 'sa', 'has']
['göm', 'hez', 'hiz']
['gö', 'nem', 'he']
['ge', 'gel', 'hiz']
['e', 'te', 'fe', 'gre']
['et', 'hiz', 'hiz']
['hav', 'hav', 'hav']
['güç', 'hez', 'hiz']
['gü', 'ger', 'ye']
['ha', 'la', 'ah', 'haz']
['ha', 'şak', 'gav']
['gün', 'hez', 'hiz']
['e', 'te', 'fe', 'gre']
['ger', 'ber', 'hem']
['ge', 'ger', 'ber']
['ge', 'gel', 'hiz']
['esp', 'hiz', 'hiz']
['e

In [20]:
word_categories = categorize(p_words)
for i in range(50):
  print(sent_generator(word_categories))

günhezhiz edhizhiz halmasalı gigerberce elesenmedi.
hadamana ehhizhizce gangshaz gülhezhizi geredegidi.
hinlele felhizhizce güşerete folkhazı hakragadı.
farkanra haşakgav gönhezhizi geçegesirce erelektti.
hafarah ehhizhizce gömhezhiz hınhazhazı esphizhizdi.
fasahas hachazhaz gübhezhizi frenhizce gehefesti.
hinlele gençgep gübhezhizi hırshazca hataratadı.
günhezhiz habhazhazca eyhizhiz fakhazhazı fanfanfandı.
farkanra eregest göçhezhizi gügeryece gönemhedi.
gesirere harharbaca eregest gönhezhizi hagazhazdı.
hafarah eregest gölhezhizi fasahamca güfehegezdi.
hadazahac gegelhiz gashazhazı fabhazhazca gehefesti.
fısfısfıs harharbaca gangshaz halmasalı güfehegezdi.
fıhasal eyhizhiz fışhazhazı harharbaca elesehepti.
gerberhem güfeherce fisefif fakhazhazı hamhakardı.
genvire harphaz gözmüşgüyi hataladaca hagazhazdı.
gesirere enremeşelce halhargav feselemi güfehegezdi.
gümhezhiz gangshaz hayhayhayı enrelektçe haphazhazdı.
gerberhem hiçhizhiz eteminheti evhizhizce geçgeçgeçti.
farkanra hachazhaz